In [5]:
#to remove warnings
import warnings
warnings.filterwarnings('ignore')

In [6]:
#importing all the important libraries
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score as acc
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score

In [7]:
#reading necessary files
names = pd.read_table('mimic_synthetic_feat.name',sep='\n',header=None)
data = pd.read_table('mimic_synthetic_train.data',sep=' ',header=None)
died = pd.read_table('mimic_synthetic_train.solution',sep='\n',header=None)
test = pd.read_table('mimic_synthetic_test.data' ,sep=' ' ,header=None)

In [8]:
name_col = names[0].tolist() #converting the name row of excel to a list

In [9]:
data.drop(0,axis=1,inplace=True) 
test.drop(0,axis=1,inplace=True)
data.columns = name_col       #assigning column names
test.columns =name_col

In [10]:
data["MARITAL_STATUS"].fillna("MARRIED", inplace = True) 
data["ICU"].fillna("MICU", inplace = True) 
data["LANGUAGE"].fillna("ENGL", inplace = True)
test["LANGUAGE"].fillna("ENGL",inplace = True)

In [11]:
#columns headings to a different lists acc to their datatypes
num_cols = []
cat_col = []

for cols in data.columns:
    if data[cols].dtype == object:
        cat_col.append(cols)
    else:
        num_cols.append(cols)

In [12]:
#columns headings to a different lists acc to their datatypes
tnum_cols = []
tcat_col = []

for cols in test.columns:
    if test[cols].dtype == object:
        tcat_col.append(cols)
    else:
        tnum_cols.append(cols)

In [13]:
#filling the missing values with median
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')
data[num_cols] = imputer.fit_transform(data[num_cols])


In [14]:
#filling missing values with median
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')
test[tnum_cols] = imputer.fit_transform(test[tnum_cols])

In [15]:
#dropping unnecessary columns
data.drop(['SUBJECT_ID', 'HADM_ID','ADMITTIME','DISCHTIME', 'LANGUAGE', 'RELIGION'], axis=1, inplace=True)
test.drop(['SUBJECT_ID', 'HADM_ID','ADMITTIME','DISCHTIME', 'LANGUAGE', 'RELIGION'], axis=1, inplace=True)

In [16]:
for name_col in data.columns:
    if data[name_col].dtypes == float:
        unique_cat = len(data[name_col].unique())
        print("Feature '{name_col}' has {unique_cat}' unqiue categories".format(
             name_col = name_col,unique_cat = unique_cat))

Feature 'AGE' has 74' unqiue categories
Feature 'HR_MIN_DAY1' has 79365' unqiue categories
Feature 'HR_MAX_DAY1' has 79462' unqiue categories
Feature 'HR_MEAN_DAY1' has 79298' unqiue categories
Feature 'SYS_BP_MIN_DAY1' has 79329' unqiue categories
Feature 'SYS_BP_MAX_DAY1' has 79246' unqiue categories
Feature 'SYS_BP_MEAN_DAY1' has 79362' unqiue categories
Feature 'DIAS_BP_MIN_DAY1' has 79511' unqiue categories
Feature 'DIAS_BP_MAX_DAY1' has 79393' unqiue categories
Feature 'DIAS_BP_MEAN_DAY1' has 79387' unqiue categories
Feature 'MEAN_BP_MIN_DAY1' has 79409' unqiue categories
Feature 'MEAN_BP_MAX_DAY1' has 79512' unqiue categories
Feature 'MEAN_BP_MEAN_DAY1' has 79138' unqiue categories
Feature 'RESP_RATE_MIN_DAY1' has 79552' unqiue categories
Feature 'RESP_RATE_MAX_DAY1' has 79563' unqiue categories
Feature 'RESP_RATE_MEAN_DAY1' has 79505' unqiue categories
Feature 'TEMP_MIN_DAY1' has 75975' unqiue categories
Feature 'TEMP_MAX_DAY1' has 75272' unqiue categories
Feature 'TEMP_MEAN_DA

In [17]:
for name_col in data.columns:
    if data[name_col].dtypes == 'object':
        unique_cat = len(data[name_col].unique())
        print("Feature '{name_col}' has {unique_cat}' unqiue categories".format(
             name_col = name_col,unique_cat = unique_cat))

Feature 'INSURANCE' has 5' unqiue categories
Feature 'MARITAL_STATUS' has 7' unqiue categories
Feature 'ETHNICITY' has 41' unqiue categories
Feature 'GENDER' has 2' unqiue categories
Feature 'ICU' has 5' unqiue categories


In [18]:
for name_col in test.columns:
    if data[name_col].dtypes == 'object':
        unique_cat = len(test[name_col].unique())
        print("Feature '{name_col}' has {unique_cat}' unqiue categories".format(
             name_col = name_col,unique_cat = unique_cat))

Feature 'INSURANCE' has 5' unqiue categories
Feature 'MARITAL_STATUS' has 7' unqiue categories
Feature 'ETHNICITY' has 39' unqiue categories
Feature 'GENDER' has 2' unqiue categories
Feature 'ICU' has 5' unqiue categories


In [19]:
data["ETHNICITY"]= data["ETHNICITY"].replace("UNKNOWN/NOT_SPECIFIED", "OTHER")
data["ETHNICITY"]= data["ETHNICITY"].replace("UNABLE_TO_OBTAIN", "OTHER")
data["ETHNICITY"]= data["ETHNICITY"].replace("PATIENT_DECLINED_TO_ANSWER", "OTHER")
data["ETHNICITY"]= data["ETHNICITY"].replace("AMERICAN_INDIAN/ALASKA_NATIVE_FEDERALLY_RECOGNIZED_TRIBE", "OTHER")
data["ETHNICITY"]= data["ETHNICITY"].replace("HISPANIC/LATINO_-_HONDURAN", "OTHER")
test["ETHNICITY"]= test["ETHNICITY"].replace("UNKNOWN/NOT_SPECIFIED", "OTHER")
test["ETHNICITY"]= test["ETHNICITY"].replace("UNABLE_TO_OBTAIN", "OTHER")
test["ETHNICITY"]= test["ETHNICITY"].replace("PATIENT_DECLINED_TO_ANSWER", "OTHER")






In [20]:
imp_ft = []
nimp_ft = []
for name_col in data.columns:
    if data[name_col].dtypes == float:
        unique_cat = len(data[name_col].unique())
        if(unique_cat>1):
            imp_ft.append(name_col)
            nimp_ft.append(name_col)

In [21]:
cimp_ft = []
for name_col in data.columns:
    if data[name_col].dtypes == 'object':
        imp_ft.append(name_col)
        cimp_ft.append(name_col)

In [22]:
len(imp_ft)

162

In [23]:
fdata=data[imp_ft]

In [24]:
ftest=test[imp_ft]

In [25]:
Gender_mapping={'M':0,'F':1}                       # convert string to numeric
fdata['GENDER']=fdata['GENDER'].map(Gender_mapping)
ftest['GENDER']=ftest['GENDER'].map(Gender_mapping)


In [26]:
fdata.to_csv("mytrain.csv")   #preprocessed training file

In [27]:
ftest.to_csv("mytest.csv")    #peprocessed test file

In [28]:
died = pd.read_table('mimic_synthetic_train.solution',sep='\n',header=None)   #solution of training data set

In [29]:
died.columns = ['DIED']

In [30]:
fdata = pd.get_dummies(fdata)   #one-hot encoding(for string variables)

In [31]:
ftest = pd.get_dummies(ftest)    #one-hot encoding

In [32]:
fdata = pd.concat([fdata,died],axis=1)    #adding died as the last column

In [33]:
from sklearn import preprocessing
#for training data
x = fdata.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
fdata = pd.DataFrame(x_scaled)              #changing the array to a Data Frame

#for test data
x = ftest.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
ftest = pd.DataFrame(x_scaled)

In [34]:
X = fdata.iloc[:,:-1]         #Train data without output class
Y = fdata.iloc[:,-1:]         #solution of train data

In [35]:
from sklearn.model_selection import train_test_split            #Split data into train & test data 
x_train_res, x_test, y_train_res, y_test = train_test_split(X, Y , test_size = 0.20, random_state = 100)

In [36]:
X=pd.DataFrame(x_train_res)
Y=pd.DataFrame(y_train_res)

In [37]:
#MODEL TESTING

In [38]:
#Logistic Regression  
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()                                    #Logistic Regression
logreg.fit(X, Y)
y_pred = logreg.predict(x_test)
print(x_test)

acc_logreg = round(balanced_accuracy_score(y_pred, y_test) * 100, 2)      # Find Accuracy
print(acc_logreg)

            0         1         2         3         4         5         6    \
7415   0.918919  0.324004  0.216563  0.280724  0.568097  0.382406  0.494286   
21709  0.662162  0.283180  0.313687  0.245871  0.526574  0.325513  0.427006   
34559  0.486486  0.346492  0.347079  0.305395  0.475056  0.243103  0.272368   
42131  0.472973  0.529984  0.464776  0.521529  0.332488  0.224909  0.215410   
39177  0.851351  0.433703  0.286814  0.395470  0.348797  0.171541  0.266640   
...         ...       ...       ...       ...       ...       ...       ...   
52864  0.378378  0.731815  0.386949  0.606857  0.477100  0.146747  0.258094   
25778  0.716216  0.361506  0.327931  0.398253  0.552405  0.340994  0.488683   
1778   0.756757  0.315476  0.142308  0.192376  0.644599  0.158027  0.385551   
21329  0.108108  0.514136  0.440290  0.403741  0.455152  0.276172  0.311530   
64290  0.864865  0.417925  0.324564  0.414842  0.278848  0.167640  0.117588   

            7         8         9    ...  201  202 

In [39]:
ty_pred = logreg.predict(ftest)

In [40]:
c = 0
for i in ty_pred:
    if i == 1:
        c+=1

In [41]:
c

87

In [42]:
over_log = pd.DataFrame(ty_pred)

In [43]:
over_log.to_csv("mimic_synthetic_test.csv")   #75.76

In [44]:
#OTHER MODELS

In [45]:
# Support Vector Machines
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV


svc = SVC()
svc.fit(x_train_res, y_train_res)
y_pred = svc.predict(x_test)
acc_logreg = round(balanced_accuracy_score(y_pred, y_test) * 100, 2)
print(acc_logreg)

96.37


In [46]:
ty_pred = svc.predict(ftest)

In [47]:
#Linear SVC
from sklearn.svm import LinearSVC

linear_svc = LinearSVC()
linear_svc.fit(x_train_res, y_train_res)

y_pred = linear_svc.predict(x_test)
acc_logreg = round(balanced_accuracy_score(y_pred, y_test) * 100, 2)
print(acc_logreg)

81.52


In [48]:
ty_pred = svc.predict(ftest)

In [49]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier

decisiontree = DecisionTreeClassifier()
decisiontree.fit(x_train_res, y_train_res)

y_pred = decisiontree.predict(x_test)
acc_logreg = round(balanced_accuracy_score(y_pred, y_test) * 100, 2)
print(acc_logreg)

53.1


In [50]:
ty_pred = decisiontree.predict(ftest)

In [51]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

randomforest =  RandomForestClassifier(n_estimators = 1000, criterion = 'entropy', random_state = 0)
randomforest.fit(x_train_res, y_train_res)

y_pred = randomforest.predict(x_test)
acc_logreg = round(balanced_accuracy_score(y_pred, y_test) * 100, 2)
print(acc_logreg)


96.37


In [52]:
ty_pred = randomforest.predict(ftest)

In [53]:
# Gradient Boosting Classifier
from sklearn.ensemble import GradientBoostingClassifier

gbk = GradientBoostingClassifier()
gbk.fit(x_train_res, y_train_res)

y_pred = gbk.predict(x_test)
acc_logreg = round(balanced_accuracy_score(y_pred, y_test) * 100, 2)
print(acc_logreg)

69.63


In [54]:
ty_pred = gbk.predict(ftest)

In [55]:
# Gaussian Naive Bayes
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

gaussian = GaussianNB()
gaussian.fit(x_train_res, y_train_res)

y_pred = gaussian.predict(x_test)
acc_logreg = round(balanced_accuracy_score(y_pred, y_test) * 100, 2)
print(acc_logreg)

50.07


In [56]:
ty_pred = gaussian.predict(ftest)

In [57]:
# Perceptron
from sklearn.linear_model import Perceptron

perceptron = Perceptron()
perceptron.fit(x_train_res, y_train_res)
y_pred = perceptron.predict(x_test)
acc_logreg = round(balanced_accuracy_score(y_pred, y_test) * 100, 2)
print(acc_logreg)

68.79


In [58]:
ty_pred = perceptron.predict(ftest)

In [59]:
c=0
for i in ty_pred:
    if i==1:
        c+=1

In [60]:
c

32